[Reference](https://gecheline.medium.com/corrupted-hard-drive-python-to-the-rescue-6feb2a9e0422)

# 1. Reproduce the directory tree in a backup iCloud directory


In [1]:
import os
hd_folder = '/Volumes/CorruptedDrive/'
dest_folder = '/where/I/want/the/backup/to/go'
# if only a subset of the folders is actually important
# save time by only backing those up
important_folders = ['Completed Songs', 'Sound Effects', 'WIPs']
os.mkdir(dest_folder)
individual_files = []
for folder in important_folders:
    print('NOW WALKING %s' % folder)
    for root, dirs, files in os.walk(hd_folder+folder):
        make_dir(root, hd_folder, dest_folder)
        for f in files:
            individual_files.append(root+'/'+f)

In [2]:
import numpy as np
np.savetxt('individual_files.txt', np.array(individual_files), fmt='%s')

# 2. Copy each individual file to its sibling location on iCloud


In [3]:
files = np.loadtxt('individual_files.txt', delimiter='\n', dtype='str')
for file in files:
    newfile = file.replace(hd_folder, dest_folder)
    flist = newfile.split('/')
    # the loop below checks whether directories exist at all depths available from the filename
    for i in range(2,len(flist)):
        print('/'.join(flist[0:i]), os.path.isdir('/'.join(flist[0:i])))
        if os.path.isdir('/'.join(flist[0:i])):
            pass
        else:
            os.mkdir('/'.join(flist[0:i]))

In [4]:
# extra code needed for raising the TimeOutException and copying
import signal
import time
 
class TimeOutException(Exception):
   pass
 
def alarm_handler(signum, frame):
    print("ALARM signal received")
    raise TimeOutException()
 
def copy_file(filename, root_dir, dest_dir):
    newfile = filename.replace(root_dir, dest_dir)
    shutil.copy2(filename, newfile)
    time.sleep(1)
 
signal.signal(signal.SIGALRM, alarm_handler)
signal.alarm(300) # I give each file 5 minutes before timeout

In [5]:
import shutil
flags = np.zeros(len(files))
i = 0 # keeps track of file list progress
j = 0 # keeps count of consecutive file not found issues
while i < len(files):
    print('COPYING %s [%s/%s]' % (files[i], i, len(files)))
    if flags[i] == 1 or os.path.exists(files[i].replace(hd_folder, dest_folder)):
        print('File exists, skipping!')
        flags[i] = 1
        i+=1
    else:
        try:
            copy_file(files[i], hd_folder, dest_folder)
            flags[i] = 1
            i+=1
            j=0 # reset counter
        except FileNotFoundError:
            if j > 1:
                print('File really not found, aborting.')
                flags[i] = -1
                i += 1
                j = 0 # reset counter
            else:
                print('File not found, retrying %s/2...' % str(j+1))
                time.sleep(120)
                i = i
                j += 1
        except IOError:
            print('I/O error, aborting')
            flags[i] = -2
            i += 1
        except TimeOutException:
            print('Timeout, aborting')
            flags[i] = -3
            i += 1
            signal.alarm(0)
        except Exception as ex:
            print(ex)
            flags[i] = -999
            i += 1 
        finally:
            signal.alarm(300)